In [1]:
#imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pickle

/root/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/root/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
/root/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [15]:
df = pd.read_parquet("./homework/1/data/yellow_tripdata_2023-01.parquet")

print("Columns:",df.shape[1])

df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

print("Standard deviation:", df.duration.std())

before = df.shape[0]
df = df[(df.duration >=1) & (df.duration <=60)]
after = df.shape[0]

print("Fraction left:", 100-(before-after)*100/before)

Columns: 19
Standard deviation: 42.59435124195458
Fraction left: 98.1220282212598


In [16]:
# Data preparation
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >=1) & (df.duration <=60)]
    
    categorical = ["PULocationID", "DOLocationID"]
    
    df[categorical] = df[categorical].astype(str)
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    return df

df_train = read_dataframe("./homework/1/data/yellow_tripdata_2023-01.parquet")
df_val = read_dataframe("./homework/1/data/yellow_tripdata_2023-02.parquet")

In [17]:
target = "duration"

categorical = ["PULocationID", "DOLocationID"]
numerical= [] # ["trip_distance"]


3.81

In [22]:

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

y_train = df_train[target].values
y_val = df_val[target].values

print("Dimensionality:",X_train.shape[1])

Dimensionality: 515


In [23]:
#Regressions
def store(m, fn):
    with open(fn, "wb") as f_out:
        pickle.dump((dv, m), f_out)

lr = LinearRegression()
lr.fit(X_train, y_train)

store(lr, "./homework/1/models/lin_reg.bin")

y_pred_train = lr.predict(X_train)
y_pred = lr.predict(X_val)

train_rmse = mean_squared_error(y_train, y_pred_train, squared=False)
val_rmse = mean_squared_error(y_val, y_pred, squared=False)

print("RMSE on train:", train_rmse)
print("RMSE on val:", val_rmse)


RMSE on train: 7.649261027815242
RMSE on val: 7.811832582145238


In [91]:
lr = Lasso()
lr.fit(X_train, y_train)

store(lr, "./homework/1/models/las_reg.bin")

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)



12.212583224318818

In [20]:
lr = Ridge()
lr.fit(X_train, y_train)

store(lr, "./homework/1/models/red_reg.bin")

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)


7.811828508220509